# YellowHat Hackathon

<strong>Brayton Sanders</strong><p>11/29 to 11/30/22</p>

<strong>Teammates:</strong><p>Chris<br>Josh</p>

_______

## Usefull Links

<a href="https://developer.apple.com/documentation/vision/vnrecognizetextrequest">Apple 'Vision' Developer Docs</a>

<a href="https://gist.github.com/RhetTbull/1c34fc07c95733642cffcd1ac587fc4c">Github for simple example of Python implementation</a>

__________

## Imports n Stuff

In [17]:
import os
import pathlib
import numpy as np
import Quartz # pip install pyobjc-framework-Quartz
import Vision # pip install pyobjc-framework-Vision
from Cocoa import NSURL
from Foundation import NSDictionary
# needed to capture system-level stderr
from wurlitzer import pipes # pip install wurlitzer

______

## Exploring Github Example Above

In [358]:
def image_to_text(img_path, lang="eng"):
    input_url = NSURL.fileURLWithPath_(img_path)

    with pipes() as (out, err):
    # capture stdout and stderr from system calls
    # otherwise, Quartz.CIImage.imageWithContentsOfURL_
    # prints to stderr something like:
    # 2020-09-20 20:55:25.538 python[73042:5650492] Creating client/daemon connection: B8FE995E-3F27-47F4-9FA8-559C615FD774
    # 2020-09-20 20:55:25.652 python[73042:5650492] Got the query meta data reply for: com.apple.MobileAsset.RawCamera.Camera, response: 0
        input_image = Quartz.CIImage.imageWithContentsOfURL_(input_url)

    vision_options = NSDictionary.dictionaryWithDictionary_({})
    vision_handler = Vision.VNImageRequestHandler.alloc().initWithCIImage_options_(
        input_image, vision_options
    )
    results = []
    handler = make_request_handler(results)
    vision_request = Vision.VNRecognizeTextRequest.alloc().initWithCompletionHandler_(handler)
    error = vision_handler.performRequests_error_([vision_request], None)

    return results

def make_request_handler(results):
    """ results: list to store results """
    if not isinstance(results, list):
        raise ValueError("results must be a list")

    def handler(request, error):
        if error:
            print(f"Error! {error}")
        else:
            # request.setMinimumTextHeight_(1/1000)
            # request.setRecognitionLevel_(1)
            
            observations = request.results()
            for text_observation in observations:
                recognized_text = text_observation.topCandidates_(1)[0]
                results.append([recognized_text.string(), recognized_text.confidence(), text_observation])
    return handler

In [359]:
# Get a png
path = os.path.join('..', "assets", "two-1.png")

os.path.exists(path)

True

In [361]:
# Process Image!
res = image_to_text(img_path = path)

# [string, confidencem, VNRecognizedTextObservation object]
# res

### Exploring <strong>"VNRecognizedTextObservation"</strong> Object

In [85]:
ob = res[0][-1]

In [86]:
# Found area where text exists
ob.boundingBox()

<CoreFoundation.CGRect origin=<CoreFoundation.CGPoint x=0.8953488365052238 y=0.8779956499735514> size=<CoreFoundation.CGSize width=0.010174433390299553 height=0.05228757858276367>>

In [89]:
# Corrdinates relative to bottom left of image
## Should be ratio -> 0,0 == bottom left OR 1,1 == top right
ob.bottomLeft()

<CoreFoundation.CGPoint x=0.9055232698955233 y=0.8779956499735514>

In [431]:
# All atributes (long list)
# np.sort(dir(ob)).tolist()

## Pull Apart Function

In [501]:
def make_request_handler(results):
    """ results: list to store results """
    if not isinstance(results, list):
        raise ValueError("results must be a list")

    def handler(request, error):
        if error:
            print(f"Error! {error}")
        else:
            request.setMinimumTextHeight_(1/2)
            # request.setRegionOfInterest_([[0, 0], [0.5, 0.5]])
            # request.setRecognitionLevel_(1)
            
            observations = request.results()
            for text_observation in observations:
                recognized_text = text_observation.topCandidates_(1)[0]
                if len(results) == 0:
                    results.append([recognized_text.string(), recognized_text.confidence(), request])
                else:
                    results.append([recognized_text.string(), recognized_text.confidence(), (text_observation.bottomLeft().x, text_observation.bottomLeft().y)])                    
    return handler

In [518]:
# Get a png
image_num = 1
path = os.path.join('..', '..', "crime_lab_roof", f"crime_plans_page_{image_num}.jpeg")


## Here I tried to split an image into smaller images so the text on the image is relatively bigger and maybe will be picked up.. not much luck

# path = os.path.join('..', '..', "crime_lab_roof", "split_image", f"row-1-column-1.jpg")

os.path.exists(path)

True

In [507]:
# VNRequestTextRecoghhhnitionLevel
# minimumTextHeight

In [508]:
try:
    del(results)
except NameError:
    pass

input_url = NSURL.fileURLWithPath_(path)

with pipes() as (out, err):
# capture stdout and stderr from system calls
# otherwise, Quartz.CIImage.imageWithContentsOfURL_
# prints to stderr something like:
# 2020-09-20 20:55:25.538 python[73042:5650492] Creating client/daemon connection: B8FE995E-3F27-47F4-9FA8-559C615FD774
# 2020-09-20 20:55:25.652 python[73042:5650492] Got the query meta data reply for: com.apple.MobileAsset.RawCamera.Camera, response: 0
    input_image = Quartz.CIImage.imageWithContentsOfURL_(input_url)

# "recognitionLevel": Vision.VNRequestTextRecognitionLevel({'fast': 0})
    
vision_options = NSDictionary.dictionaryWithDictionary_({})
vision_handler = Vision.VNImageRequestHandler.alloc().initWithCIImage_options_(
    input_image, vision_options
)
results = []
handler = make_request_handler(results)
vision_request = Vision.VNRecognizeTextRequest.alloc().initWithCompletionHandler_(handler)
error = vision_handler.performRequests_error_([vision_request], None)

In [512]:
'''
This request object *seems* to be where you set the "minimumTextHeight" arg

But you can set it to 1/1000 or 1/2 and the results don't change...
''';

req = results[0][-1]
# dir(results[0][-1])

In [514]:
req.minimumTextHeight()

0.5

In [515]:
# Default Case: minimumTextHeight = 1/32
print(f'Found: {len(results)} observations\n')
results

Found: 11



[['VAN',
  0.5,
  <VNRecognizeTextRequest: 0x600002eedd80> VNRecognizeTextRequestRevision3 ROI=[0, 0, 1, 1]],
 ['OVERALL ROOF PLAN', 1.0, (0.815406982421875, 0.04575162760416662)],
 ['A100', 0.5, (0.9244185926649305, 0.037037027994791694)],
 ['OVERALL ROOF', 1.0, (0.9286655002170139, 0.1098664143880208)],
 ['EASTERN REGIONAL CRIME', 1.0, (0.9360465223524306, 0.2723311564127604)],
 ['LABORATORY REROOF', 0.5, (0.9491279025607638, 0.29411763509114586)],
 ['GEORGIA BUREAU OF INVESTIGATION',
  1.0,
  (0.9622093098958333, 0.2832244059244792)],
 ['1945 PHINIZY RD., AUGUSTA, GA 30906',
  0.5,
  (0.9723837348090277, 0.2962963053385417)],
 ['MERIK', 0.5, (0.9447674424913195, 0.6753812662760417)],
 ['3225 SHALLOWFORD ROAD', 0.5, (0.9418604871961805, 0.871459716796875)],
 ['BUILDING CONSTRUCTION CONSULTANTS',
  0.5,
  (0.9680232747395834, 0.67320263671875)]]

In [516]:
results[1]

['OVERALL ROOF PLAN', 1.0, (0.815406982421875, 0.04575162760416662)]

In [517]:
# I have to be missing something somewhere.....